## [Moreau et al. (2021)](https://hal.archives-ouvertes.fr/hal-03346165/document)
### What is done:
* delete unlabeled data
* join car and taxi into one category
* delete unwanted labels
* compute speed and acceleration
* remove unrealistic data points
<img src="Tresholds for transport modes.PNG" width="800" height="400">
* interpolate linearly the data points (T=2s)
* merge all users and divide into trips ($\Delta$t <= 20min)
* splitting trips: 
 * 64% training
 * 16% validation
 * 20% test (pseudocode)

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

### Passo 1

In [2]:
import numpy as np
import pandas as pd
import glob
import os.path
import datetime
import os

def read_plt(plt_file):
    points = pd.read_csv(plt_file, skiprows=6, header=None,
                         parse_dates=[[5, 6]], infer_datetime_format=True)

    # for clarity rename columns
    points.rename(inplace=True, columns={'5_6': 'time', 0: 'lat', 1: 'lon', 3: 'alt'})

    # remove unused columns
    points.drop(inplace=True, columns=[2, 4])

    return points

# mode_names = ['walk', 'bike', 'bus', 'car', 'subway','train', 'airplane', 'boat', 'run', 'motorcycle', 'taxi']
mode_ids = {'walk':1, 'bike':2, 'bus':3, 'car':4, 'taxi':4, 'train':5, 'subway':6, 'airplane':0, 'boat':0, 'run':0, 'motorcycle':0}

def read_labels(labels_file):
    labels = pd.read_csv(labels_file, skiprows=1, header=None,
                         parse_dates=[[0, 1], [2, 3]],
                         infer_datetime_format=True, delim_whitespace=True)

    # for clarity rename columns
    labels.columns = ['start_time', 'end_time', 'label']

    # replace 'label' column with integer encoding
    labels['label'] = [mode_ids[i] for i in labels['label']]

    return labels

def apply_labels(points, labels):
    indices = labels['start_time'].searchsorted(points['time'], side='right') - 1
    no_label = (indices < 0) | (points['time'].values >= labels['end_time'].iloc[indices].values)
    points['label'] = labels['label'].iloc[indices].values
    points['label'][no_label] = 0

def read_user(user_folder):
    labels = None

    plt_files = glob.glob(os.path.join(user_folder, 'Trajectory', '*.plt'))
    df = pd.concat([read_plt(f) for f in plt_files])

    labels_file = os.path.join(user_folder, 'labels.txt')
    if os.path.exists(labels_file):
        labels = read_labels(labels_file)
        apply_labels(df, labels)
    else:
        df['label'] = 0

    return df

def read_all_users(folder):
    subfolders = os.listdir(folder)
    dfs = []
    for i, sf in enumerate(subfolders):
        print('[%d/%d] processing user %s' % (i + 1, len(subfolders), sf))
        df = read_user(os.path.join(folder,sf))
        df['user'] = int(sf)
        dfs.append(df)
    return pd.concat(dfs)

In [3]:
start = datetime.datetime.now()
df1 = read_all_users('../Data')
df1
end = datetime.datetime.now()
print('Tempo gasto no passo 1:',end-start)

[1/182] processing user 000
[2/182] processing user 001
[3/182] processing user 002
[4/182] processing user 003
[5/182] processing user 004
[6/182] processing user 005
[7/182] processing user 006
[8/182] processing user 007
[9/182] processing user 008
[10/182] processing user 009
[11/182] processing user 010
[12/182] processing user 011
[13/182] processing user 012
[14/182] processing user 013
[15/182] processing user 014
[16/182] processing user 015
[17/182] processing user 016
[18/182] processing user 017
[19/182] processing user 018
[20/182] processing user 019
[21/182] processing user 020
[22/182] processing user 021
[23/182] processing user 022
[24/182] processing user 023
[25/182] processing user 024
[26/182] processing user 025
[27/182] processing user 026
[28/182] processing user 027
[29/182] processing user 028
[30/182] processing user 029
[31/182] processing user 030
[32/182] processing user 031
[33/182] processing user 032
[34/182] processing user 033
[35/182] processing use

### Passo 2

In [4]:
# delete unlabeled data
# join car and taxi into one category
# delete unwanted labels

df2 = df1[df1['label'] != 0]
df2.reset_index(inplace=True)
df2.drop('index', axis=1, inplace=True)
df2

,time,lat,lon,alt,label,user
0,2008-03-28 14:54:40,39.894178,116.318200,-777.0,5,10
1,2008-03-28 14:55:14,39.894505,116.321132,-777.0,5,10
2,2008-03-28 14:56:13,39.894953,116.326452,-777.0,5,10
3,2008-03-28 14:57:12,39.894600,116.332542,-777.0,5,10
4,2008-03-28 14:58:11,39.889622,116.337040,-777.0,5,10
...,...,...,...,...,...,...
5412061,2008-11-29 02:29:27,40.029529,116.411977,291.0,6,179
5412062,2008-11-29 02:29:29,40.029320,116.411975,289.0,6,179
5412063,2008-11-29 02:29:31,40.029111,116.411963,275.0,6,179
5412064,2008-11-29 02:29:33,40.028904,116.411962,274.0,6,179


In [5]:
# # a normalização altera mto os resultados?

# from sklearn import preprocessing

# values = df2['lat'].values.reshape(-1, 1)
# min_max_scaler = preprocessing.MinMaxScaler((-90,90))
# scaled = min_max_scaler.fit_transform(values)
# df2['lat'] = pd.DataFrame(scaled)

# df2

### Passo 3

In [6]:
# compute speed and acceleration

# from geopy.distance import distance
import haversine as hs
from haversine import Unit
from geographiclib.geodesic import Geodesic
import datetime

def get_features(df):
    users = df['user'].values
    
    # getting distances between points
    dist = []
    lat = df['lat'].values
    lon = df['lon'].values
    for i in range(len(users)):
        if i != 0 and users[i-1]==users[i]:
            coords_1 = (lat[i-1],lon[i-1])
            coords_2 = (lat[i],lon[i])
#             dist.append(distance(coords_1, coords_2).km*1000)
            dist.append(hs.haversine(coords_1,coords_2,unit=Unit.METERS))
        else:
            dist.append(0)
    df['distance'] = dist
    print('........................Finished distance......................')
    
    # getting time passed between points
    timestamps = df['time'].values
    delta_time = []
    for i in range(len(users)):
        if i != 0 and users[i-1]==users[i]:
            start = timestamps[i-1]
            end = timestamps[i]
            delta_time.append(pd.Timedelta((end - start)).total_seconds())
        else:
            delta_time.append(0)
    df['delta_time'] = delta_time
    print('........................Finished time......................')
    
    # getting speed of vehicles between points
    speeds = []
    for i in range(len(users)):
        if i != 0 and users[i-1]==users[i] and dist[i] != 0 and delta_time[i] != 0:
            speeds.append(dist[i]/delta_time[i])
        else:
            speeds.append(0)
    df['speed'] = speeds
    print('........................Finished speed......................')
    
    # getting acceleration of vehicles between points
    accs = []
    for i in range(len(users)):
        if i != 0 and users[i-1]==users[i] and speeds[i] != 0 and delta_time[i] != 0:
            accs.append(speeds[i]/delta_time[i])
        else:
            accs.append(0)
    df['acceleration'] = accs
    print('........................Finished acceleration......................')
    
    return df

df3 = df2
start = datetime.datetime.now()
get_features(df3)
end = datetime.datetime.now()
print('Tempo gasto no passo 3:',end-start)
df3

........................Finished distance......................
........................Finished time......................
........................Finished speed......................
........................Finished acceleration......................
Tempo gasto no passo 3: 0:01:38.373226


,time,lat,lon,alt,label,user,distance,delta_time,speed,acceleration
0,2008-03-28 14:54:40,39.894178,116.318200,-777.0,5,10,0.000000,0.0,0.000000,0.000000
1,2008-03-28 14:55:14,39.894505,116.321132,-777.0,5,10,252.763857,34.0,7.434231,0.218654
2,2008-03-28 14:56:13,39.894953,116.326452,-777.0,5,10,456.583141,59.0,7.738697,0.131164
3,2008-03-28 14:57:12,39.894600,116.332542,-777.0,5,10,521.027624,59.0,8.830977,0.149678
4,2008-03-28 14:58:11,39.889622,116.337040,-777.0,5,10,673.539517,59.0,11.415924,0.193490
...,...,...,...,...,...,...,...,...,...,...
5412061,2008-11-29 02:29:27,40.029529,116.411977,291.0,6,179,23.352364,2.0,11.676182,5.838091
5412062,2008-11-29 02:29:29,40.029320,116.411975,289.0,6,179,23.240396,2.0,11.620198,5.810099
5412063,2008-11-29 02:29:31,40.029111,116.411963,275.0,6,179,23.262221,2.0,11.631110,5.815555
5412064,2008-11-29 02:29:33,40.028904,116.411962,274.0,6,179,23.017539,2.0,11.508770,5.754385


In [7]:
len(df3['user'].values)

5412066

### Passo 4

In [8]:
# remove unrealistic data points

start = datetime.datetime.now()


df4 = df3[((df3['label']==1) & (df3['speed']<=7) &
                          (df3['acceleration']<=3)) |
                          
                         ((df3['label']==2) & (df3['speed']<=12) &
                          (df3['acceleration']<=3)) |
                         
                         ((df3['label']==3) & (df3['speed']<=34) &
                          (df3['acceleration']<=2)) |
                         
                         ((df3['label']==4) & (df3['speed']<=50) &
                          (df3['acceleration']<=10)) |
                         
                         ((df3['label']==5) & (df3['speed']<=34) &
                          (df3['acceleration']<=3))]


end = datetime.datetime.now()
print('Tempo gasto no passo 4:',end-start)
df4.reset_index(inplace=True)
df4.drop('index', axis=1, inplace=True)
df4

Tempo gasto no passo 4: 0:00:00.664257


,time,lat,lon,alt,label,user,distance,delta_time,speed,acceleration
0,2008-03-28 14:54:40,39.894178,116.318200,-777.0,5,10,0.000000,0.0,0.000000,0.000000
1,2008-03-28 14:55:14,39.894505,116.321132,-777.0,5,10,252.763857,34.0,7.434231,0.218654
2,2008-03-28 14:56:13,39.894953,116.326452,-777.0,5,10,456.583141,59.0,7.738697,0.131164
3,2008-03-28 14:57:12,39.894600,116.332542,-777.0,5,10,521.027624,59.0,8.830977,0.149678
4,2008-03-28 14:58:11,39.889622,116.337040,-777.0,5,10,673.539517,59.0,11.415924,0.193490
...,...,...,...,...,...,...,...,...,...,...
3680261,2008-11-29 02:04:05,40.069559,116.313143,80.0,1,179,3.353172,2.0,1.676586,0.838293
3680262,2008-11-29 02:04:07,40.069541,116.313136,80.0,1,179,2.088265,2.0,1.044133,0.522066
3680263,2008-11-29 02:04:09,40.069527,116.313129,80.0,1,179,1.666798,2.0,0.833399,0.416700
3680264,2008-11-29 02:04:11,40.069515,116.313123,80.0,1,179,1.428684,2.0,0.714342,0.357171


### Passo 5 (???)

In [9]:
# interpolate

df5 = df4[['lat','lon','alt','label','user','distance','delta_time','speed','acceleration']]
df5.interpolate()
df5

,lat,lon,alt,label,user,distance,delta_time,speed,acceleration
0,39.894178,116.318200,-777.0,5,10,0.000000,0.0,0.000000,0.000000
1,39.894505,116.321132,-777.0,5,10,252.763857,34.0,7.434231,0.218654
2,39.894953,116.326452,-777.0,5,10,456.583141,59.0,7.738697,0.131164
3,39.894600,116.332542,-777.0,5,10,521.027624,59.0,8.830977,0.149678
4,39.889622,116.337040,-777.0,5,10,673.539517,59.0,11.415924,0.193490
...,...,...,...,...,...,...,...,...,...
3680261,40.069559,116.313143,80.0,1,179,3.353172,2.0,1.676586,0.838293
3680262,40.069541,116.313136,80.0,1,179,2.088265,2.0,1.044133,0.522066
3680263,40.069527,116.313129,80.0,1,179,1.666798,2.0,0.833399,0.416700
3680264,40.069515,116.313123,80.0,1,179,1.428684,2.0,0.714342,0.357171


### Passo 6

In [10]:
# merge users and split into trips

df6 = df5
delta_time = df5['delta_time'].values
trips = [1]
t = 1
for i in range(1,len(delta_time)):
    if delta_time[i] <= 1200:
        trips.append(t)
    else:
        t += 1
        trips.append(t)
df6['trips'] = trips
df6.drop(['user'], axis = 1, inplace = True)
df6

,lat,lon,alt,label,distance,delta_time,speed,acceleration,trips
0,39.894178,116.318200,-777.0,5,0.000000,0.0,0.000000,0.000000,1
1,39.894505,116.321132,-777.0,5,252.763857,34.0,7.434231,0.218654,1
2,39.894953,116.326452,-777.0,5,456.583141,59.0,7.738697,0.131164,1
3,39.894600,116.332542,-777.0,5,521.027624,59.0,8.830977,0.149678,1
4,39.889622,116.337040,-777.0,5,673.539517,59.0,11.415924,0.193490,1
...,...,...,...,...,...,...,...,...,...
3680261,40.069559,116.313143,80.0,1,3.353172,2.0,1.676586,0.838293,5964
3680262,40.069541,116.313136,80.0,1,2.088265,2.0,1.044133,0.522066,5964
3680263,40.069527,116.313129,80.0,1,1.666798,2.0,0.833399,0.416700,5964
3680264,40.069515,116.313123,80.0,1,1.428684,2.0,0.714342,0.357171,5964


### Passo 7

In [11]:
# split into triplegs

df7 = df6
trips = df5['trips'].values
labels = df5['label'].values
triplegs = [1]
t = 1
for i in range(1,len(trips)):
    if trips[i]==trips[i-1] and labels[i]==labels[i-1]:
        triplegs.append(t)
    else:
        t+=1
        triplegs.append(t)
df7['triplegs'] = triplegs
df7

,lat,lon,alt,label,distance,delta_time,speed,acceleration,trips,triplegs
0,39.894178,116.318200,-777.0,5,0.000000,0.0,0.000000,0.000000,1,1
1,39.894505,116.321132,-777.0,5,252.763857,34.0,7.434231,0.218654,1,1
2,39.894953,116.326452,-777.0,5,456.583141,59.0,7.738697,0.131164,1,1
3,39.894600,116.332542,-777.0,5,521.027624,59.0,8.830977,0.149678,1,1
4,39.889622,116.337040,-777.0,5,673.539517,59.0,11.415924,0.193490,1,1
...,...,...,...,...,...,...,...,...,...,...
3680261,40.069559,116.313143,80.0,1,3.353172,2.0,1.676586,0.838293,5964,9279
3680262,40.069541,116.313136,80.0,1,2.088265,2.0,1.044133,0.522066,5964,9279
3680263,40.069527,116.313129,80.0,1,1.666798,2.0,0.833399,0.416700,5964,9279
3680264,40.069515,116.313123,80.0,1,1.428684,2.0,0.714342,0.357171,5964,9279


### Passo 8

In [12]:
# start = datetime.datetime.now()

# trplgs = []
# unique_trips = df7['triplegs'].unique()
# for u in unique_trips:
#     tripleg = df7[df7['triplegs']==u].values.tolist()
#     trplgs.append(tripleg)
# trplgs = np.array(trplgs)

# end = datetime.datetime.now()
# print('Tempo gasto no passo 8:',end-start)

Na seção 4.1.2 é explicado que a base é dividida por triplegs ao invés de usuários ao se criar os sets de train, validation e test, para manter a distribuição dos tipos de transporte. Já no pseudo-código os dados são divididos em trips antes. Isso não causaria o mesmo problema de não ter como garantir a distribuição? A única vantagem seria manter a integridade das trips em cada grupo. Como modelos shallow não identificam ordem ou tem memória dos valores anteriores, foi decidido que nesse processo a base será dividida por triplegs, a fim de replicar as quantidades de dados nos sets de train, validation e test.

In [13]:
import seaborn as sns

visualizing_df = df7[['alt','distance','delta_time','speed','acceleration','label']].sample(frac=0.2, replace=True, random_state=1)
visualizing_df.reset_index(inplace=True)
visualizing_df.drop('index', axis=1, inplace=True)

# sns.pairplot(visualizing_df, hue="label")
sns.heatmap(visualizing_df, annot=True)

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x0000028C859D58B0> (for post_execute):


KeyboardInterrupt: 

### Passo 9

In [14]:
from sklearn.model_selection import train_test_split

X = df7[['lat','lon','alt','distance','delta_time','speed','acceleration','trips','triplegs']]
y = df7[['label']].squeeze().tolist()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

### Passo 10.1

#### DecisionThree / No feature selection

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

start = datetime.datetime.now()

depths = [2,3,5]
for d in depths:
    clf = DecisionTreeClassifier(max_depth=d)
    clf.fit(X_train, y_train)
    scores = cross_val_score(clf,X_test,y_test)
    print(scores)
    
end = datetime.datetime.now()
print('Tempo gasto:',end-start)

[0.57231457 0.57253194 0.57322483 0.57377506 0.57407785]
[0.5777897  0.57763346 0.5790532  0.57879506 0.57945112]
[0.66378192 0.66216519 0.6617712  0.66257956 0.66295089]
Tempo gasto: 0:01:11.156735


#### Random Forest / No feature selection

In [24]:
# from sklearn.ensemble import RandomForestClassifier

# start = datetime.datetime.now()

# depths = [2,3,5]
# for d in depths:
#     clf = RandomForestClassifier(max_depth=d)
#     clf.fit(X_train, y_train)
#     scores = cross_val_score(clf,X_test,y_test)
#     print(scores)
    
# end = datetime.datetime.now()
# print('Tempo gasto:',end-start)

[0.58869548 0.59335428 0.53127848 0.59176696 0.59462565]
[0.62469786 0.61839744 0.61637655 0.61778958 0.61994067]
[0.68420933 0.68420933 0.68377912 0.69012307 0.68787007]
Tempo gasto: 0:38:25.501836


#### GBoosting / No feature selection

In [25]:
# from sklearn.ensemble import GradientBoostingClassifier

# start = datetime.datetime.now()

# estimators = [10,20,100]
# for e in estimators:
#     clf = GradientBoostingClassifier(n_estimators=e, learning_rate=1.0,
#                         max_depth=1, random_state=0).fit(X_train, y_train)
#     print(clf.score(X_test, y_test))

# end = datetime.datetime.now()
# print('Tempo gasto:',end-start)

0.6447247906366435
0.6677652322668076
0.6911385268201338
Tempo gasto: 9:20:07.233282


### Passo 10.2

#### Feature Selection - Chi-square

In [16]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import f_classif

# X_new = SelectKBest(f_classif, k=5).fit_transform(X,y)

# X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(
#     X_new, y, test_size=0.2, random_state=42)

#### Feature Selection - Variance Threshold

In [23]:
# from sklearn.feature_selection import VarianceThreshold

# sel = VarianceThreshold(threshold=(.08))
# X_new = sel.fit_transform(X)

# X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(
#     X_new, y, test_size=0.2, random_state=42)

#### Feature selection - Sequential Feature Selector

In [ ]:
# from mlxtend.feature_selection import SequentialFeatureSelector as sfs
# from sklearn.linear_model import LinearRegression

# lreg = LinearRegression()
# sfs1 = sfs(lreg, k_features=4, forward=True, verbose=2, scoring='neg_mean_squared_error')

# sfs1 = sfs1.fit(X, y)
# feat_names = list(sfs1.k_feature_names_)
# sfs_df = df7[feat_names]
########## <<<<<<<<<<<<<<<<<<<<<<<<

#### DecisionTree / With feature selection

In [25]:
start = datetime.datetime.now()

depths = [2,3,5]
for d in depths:
    clf = DecisionTreeClassifier(max_depth=d)
    clf.fit(X_train_new, y_train_new)
    scores = cross_val_score(clf,X_test_new,y_test_new)
    print(scores)
    
end = datetime.datetime.now()
print('Tempo gasto:',end-start)

[0.57231457 0.57253194 0.57322483 0.57377506 0.57407785]
[0.5777897  0.57763346 0.5790532  0.57879506 0.57945112]
[0.66378871 0.66216519 0.6617712  0.66257956 0.66295089]
Tempo gasto: 0:01:13.709504


Normal

[0.57231457 0.57253194 0.57322483 0.57377506 0.57407785]

[0.5777897  0.57763346 0.5790532  0.57879506 0.57945112]

[0.66378192 0.66216519 0.6617712  0.66257956 0.66295089]

Tempo gasto: 0:01:11.156735

Chi-Squared

[0.57231457 0.57253194 0.57322483 0.57377506 0.57407785]

[0.58609071 0.58601599 0.5869738  0.58641678 0.58740575]

[0.66275618 0.66143834 0.6621584  0.66145193 0.66341281]

Tempo gasto: 0:00:51.462330

Variance .5

[0.57231457 0.57253194 0.57322483 0.57377506 0.57407785]

[0.5777897  0.57763346 0.5790532  0.57879506 0.57945112]

[0.66378192 0.66216519 0.6617712  0.66257956 0.66295089]

Tempo gasto: 0:01:14.069283

#### Random Forest / With feature selection

In [28]:
# from sklearn.ensemble import RandomForestClassifier

# start = datetime.datetime.now()

# depths = [2,3,5]
# for d in depths:
#     clf = RandomForestClassifier(max_depth=d)
#     clf.fit(X_train_new, y_train_new)
#     scores = cross_val_score(clf,X_test_new,y_test_new)
#     print(scores)
    
# end = datetime.datetime.now()
# print('Tempo gasto:',end-start)

[0.56347117 0.55899918 0.57413037 0.55771168 0.56162329]
[0.60740426 0.60470408 0.60320399 0.60949653 0.60090346]
[0.65378846 0.65018256 0.65316011 0.65705843 0.65348082]
Tempo gasto: 0:32:32.034048


#### GBoosting / With feature selection

In [29]:
# from sklearn.ensemble import GradientBoostingClassifier

# start = datetime.datetime.now()

# estimators = [10,20,100]
# for e in estimators:
#     clf = GradientBoostingClassifier(n_estimators=e, learning_rate=1.0,
#                         max_depth=1, random_state=0).fit(X_train_new, y_train_new)
#     print(clf.score(X_test_new, y_test_new))

# end = datetime.datetime.now()
# print('Tempo gasto:',end-start)

0.6072618544889292
0.6260963484675746
0.6614240444347331
Tempo gasto: 0:33:19.759216
